# Representing Data in iCn3D

In this notebook I'm in implementing 3 classes to output iCn3D commands for representing 3 types of data:

* Covalent Labelling
* Hydrogen Deuterium Exchange (HDX)
* DOPE quality score

In [39]:
import os
import webbrowser
from modeller import *
from modeller.scripts import complete_pdb
import urllib
import matplotlib
import matplotlib.cm as cm

In [101]:
class LoadModel:
    """
    Overarching class with methods common to all 3 sub classes
    """
    def __init__(self,pdb_code):
        self.pdb_code = pdb_code
        self.state = False
        self.url_warning=None
        
    def open_url(self,data_label=None,print_out=False,open_link=True):
        if not self.state:
            self.output_statefile(data_label=data_label,output_bool=False)
        url = f"https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html?divid=div0&pdbid={self.pdb_code}&command="
        for line in self.state:
            if "mmdb" not in line:
                url+=f"{line.replace(' ','%20')};"
        
        if open_link:
            if not self.url_warning:
                webbrowser.open(url)
            else:
                print(self.url_warning)
        
        if print_out:
            print(url)
            
    def output(self,lines,state_pref,state_suff):
        with open("%s%s%s" % (state_pref,self.pdb_code,state_suff),"w+") as f:
                for line in lines:
                    f.write(f"{line}\n")
                f.close()

class RepLabels(LoadModel):
    """
    A class to output iCn3D commands for surface labelling information
    """
    def __init__(self,pdb_code,labels):
        super().__init__(pdb_code)
        self.labels = self.get_labels(labels)
        
        
    def get_labels(self,labels):
        """Parse covalent labels and return residue ids"""
        mls = set()
        
        for line in (open(labels)).read().splitlines():
            ml = line.split('|')
            mls.add((int(ml[0]),ml[1]))
            
        return mls
    
    def output_statefile(self,data_label="",state_pref="./",state_suff="_state.txt",color="0FF",output_bool=True):
        lines = []

        lines.append(f"load mmdb {self.pdb_code}")

        for ml in self.labels:
            lines.append(f"select .{ml[1]}:{ml[0]}" )
            lines.append("style sidec ball and stick")
            lines.append(f"color {color}")
        
        if output_bool:
            self.output(lines,state_pref,state_suff)
            
        self.state = lines
        
class HDXRepr(LoadModel):
    """
    A class to output iCn3D commands for representing HDX information.
    HDX format is as gathered from start2fold database
    """
    def __init__(self,pdb_code,hdx_file):
        super().__init__(pdb_code)
        self.stability,self.folding = self.read_hdx(hdx_file)
        self.cols = {
            "EARLY":"green",
            "STRONG":"green",
            "INTERMEDIATE":"yellow",
            "MEDIUM":"yellow",
            "WEAK":"red",
            "LATE":"red"
        }
        
    def read_hdx(self,hdx_file):
        stability = {}
        folding = {}
        
        with open(hdx_file) as f:
            g = f.read().splitlines()
            for line in [i for i in g if not i.startswith("#")]:
                state = line.split(';')[-1].strip()
                aa = int(line.split(';')[0].strip())
                if state in ["EARLY","INTERMEDIATE","LATE"]:
                    folding[aa] = state
                else:
                    stability[aa] = state
            f.close()
            
        return stability, folding
    
    def output_statefile(self, data_label="stability", output_bool=True, state_pref="./",state_suff="_state.txt"):
        lines = []
        
        if data_label.lower() == "stability":
            d = self.stability
        elif data_label.lower() == "folding":
            d = self.folding
        else:
            print("Only stability and folding are allowed options.")
            print("No state file output.")
            return
        
        lines.append(f"load mmdb {self.pdb_code}")
        
        lines.append("color silver")
        
        for k,v in d.items():
            lines.append(f"select :{k}")
            lines.append(f"color {self.cols[v]}")
            
        if output_bool:
            self.output(lines,state_pref, state_suff)
            
        self.state = lines
        
class DOPE(LoadModel):
    """A class to output iCn3D commands for representing per-residue DOPE scores in"""
    def __init__(self,pdb_code,pdb_loc="."):
        super().__init__(pdb_code)
        self.pdb_loc = pdb_loc
        self.get_pdb()
        self.res_ids = self.get_res_ids()
        self.dope_scores = self.score_dope()
        self.url_warning = "DOPE scoring urls are too long to be accessed. Please output a state file and load into iCn3D."
        
    def get_pdb(self):
        try:
            os.mkdir(self.pdb_loc)
        except:
            pass
        
        urllib.request.urlretrieve(f"http://files.rcsb.org/download/{self.pdb_code}.pdb", f"{self.pdb_loc}/{self.pdb_code}.pdb")
    
    def get_res_ids(self):
        res_ids = set()
        with open(f"{self.pdb_loc}/{self.pdb_code}.pdb") as f:
            g = f.read().splitlines()
            for line in [i for i in g if i.startswith("ATOM")]:
                res_ids.add(int(line[22:28].strip()))
            f.close()
            
        return res_ids
    
    def score_dope(self):
        
        dope_scores = []
        
        env = environ()
        env.libs.topology.read(file='$(LIB)/top_heav.lib') # read topology
        env.libs.parameters.read(file='$(LIB)/par.lib') # read parameters
        
        mdl = complete_pdb(env, f"{self.pdb_loc}/{self.pdb_code}.pdb")
        
        # Assess with DOPE:
        s = selection(mdl)   # all atom selection
        s.assess_dope(output='ENERGY_PROFILE NO_REPORT', file=f"{self.pdb_loc}/{self.pdb_code}.profile",
                      normalize_profile=True, smoothing_window=15)
        
        with open(f"{self.pdb_loc}/{self.pdb_code}.profile") as f:
            g = f.read().splitlines()
            for line in [i for i in g[7:]]:
                dope_scores.append(float(line.split()[-1]))
            f.close()
            
        return {k:v for k,v in zip(self.res_ids,dope_scores)}
        
    def output_statefile(self, data_label="", output_bool=True, state_pref="./",state_suff="_state.txt"):
        lines = []
        
        lines.append(f"load mmdb {self.pdb_code}")
        
        lines.append("color silver")
        
        minima = min(self.dope_scores.values())
        maxima = max(self.dope_scores.values())
        norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
        mapper = cm.ScalarMappable(norm=norm, cmap=cm.Reds_r)
        
        dope_colors = {k:matplotlib.colors.rgb2hex(mapper.to_rgba(v)) for k,v in self.dope_scores.items()}
        
#         print(dope_colors)
        
        for k,v in dope_colors.items():
            lines.append(f"select :{k}")
            lines.append(f"color {v}")
            
        if output_bool:
            self.output(lines,state_pref, state_suff)
            
        self.state = lines
            
class CustomRes(LoadModel):
    def __init__(self,pdb_code,data_file):
        super().__init__(pdb_code)
        self.res_dict = self.get_res_score(data_file)
        
    def get_res_score(self,data_file):
        d_scores = {}
        
        with open(data_file) as f:
            g = f.read().splitlines()
            for line in g:
                aa, c, score = line.split('|')
                
                d_scores[(aa,c)] = float(score)
            f.close()
            
        return d_scores
    
    def output_statefile(self, data_label="", output_bool=True, state_pref="./",state_suff="_state.txt"):
        lines = []
        
        lines.append(f"load mmdb {self.pdb_code}")
        
        lines.append("color silver")
        
        minima = min(self.res_dict.values())
        maxima = max(self.res_dict.values())
        norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
        mapper = cm.ScalarMappable(norm=norm, cmap=cm.Reds_r)
        
        print(self.res_dict)
        
        res_colors = {k:matplotlib.colors.rgb2hex(mapper.to_rgba(v)) for k,v in self.res_dict.items()}
        
        print(res_colors)
        
        for k,v in res_colors.items():
            lines.append(f"select .{k[1]}:{k[0]}")
            lines.append(f"color {v[1:]}")
            
        if output_bool:
            self.output(lines,state_pref, state_suff)
            
        self.state = lines

# Representing Covalent Labelling Information

One of the ideas we had was a method to represent residues from something like covalent labelling approaches for idenitfying surface residues. This gives a simple implementation for that.

This uses the example of Creatine Kinase (which I had covalent labelling data for), PDB:2CRK.

In [91]:
rep_lab = RepLabels("2crk","./covalent_labels/ml.txt")
rep_lab.output_statefile(state_pref="./covalent_labels/")
rep_lab.open_url(print_out=True)

https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html?divid=div0&pdbid=2crk&command=select%20.A:224;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:158;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:40;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:381;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:125;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:138;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:223;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:128;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:141;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:108;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:105;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:16;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:298;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:170;style%20sidec%20ball%20and%20stick;color%200FF;select%20.A:101;style%20sidec%20bal

When the resulting statefile is loaded into icn3d it looks like this:

![Sreeenshot](covalent_labels/screenshot.png)

# Representing Hydrogen Deuterium Exchange Data
This cell uses the HDXRepr class to parse HDX results files from the Start2Fold database (http://bio2byte.be/start2fold/) and colour residues based on their hydrogen exchange properites.

This data can provide information on the relative stability of residues and their relative exposure (see website for more info), this can be useful with the dynamical information which we were discussing.

In [59]:
hdx_mod = HDXRepr("1am7","./hdx/hdx_results.txt")
hdx_mod.open_url("stability",print_out=True)

https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html?divid=div0&pdbid=1am7&command=color%20silver;select%20:11;color%20green;select%20:12;color%20green;select%20:14;color%20green;select%20:15;color%20green;select%20:17;color%20green;select%20:18;color%20green;select%20:19;color%20green;select%20:21;color%20green;select%20:35;color%20green;select%20:67;color%20green;select%20:69;color%20green;select%20:92;color%20green;select%20:93;color%20green;select%20:94;color%20green;select%20:96;color%20green;select%20:97;color%20green;select%20:98;color%20green;select%20:99;color%20green;select%20:100;color%20green;select%20:108;color%20green;select%20:117;color%20green;select%20:120;color%20green;select%20:9;color%20yellow;select%20:10;color%20yellow;select%20:16;color%20yellow;select%20:20;color%20yellow;select%20:33;color%20yellow;select%20:36;color%20yellow;select%20:42;color%20yellow;select%20:64;color%20yellow;select%20:65;color%20yellow;select%20:66;color%20yellow;select%20:68;color%20ye

# Representing DOPE score for structure quality

This cell uses the DOPE class to calculate per residue DOPE scores using MODELLER. This part requires a local download so it fetches the pdb file from the PDB and performs the scoring locally. It then maps the values to a colour gradient which is used to produce iCn3D commands.

Important to note: although the other two methods produced useable URLs to easily open the iCn3D entry, this method produces a url which is too long to open. We therefore need to output a state file and load it manually into iCn3D.

In [54]:
dope_mod = DOPE("2crk")
dope_mod.output_statefile()

read_to_681_> topology.submodel read from topology file:        3
>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  LYS:OXT  LYS
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      365
Number of all, selected real atoms                :     2928    2928
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   590896
Dynamic pairs routine                             : 1, NATM x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH      

In [75]:
dope_mod.open_url(open_link=True,print_out=True)

https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html?divid=div0&pdbid=2crk&command=load%20mmdb%202crk;color%20silver;select%20:8;color%20#fed9c9;select%20:9;color%20#fdc6b0;select%20:10;color%20#fcb499;select%20:11;color%20#fcaa8d;select%20:12;color%20#fc9576;select%20:13;color%20#fc9576;select%20:14;color%20#fc8d6d;select%20:15;color%20#fc8262;select%20:16;color%20#fc8d6d;select%20:17;color%20#fc8d6d;select%20:18;color%20#fc9576;select%20:19;color%20#fc9576;select%20:20;color%20#fc8d6d;select%20:21;color%20#fc8d6d;select%20:22;color%20#fc8d6d;select%20:23;color%20#fc8262;select%20:24;color%20#fc8262;select%20:25;color%20#fc8262;select%20:26;color%20#fc8d6d;select%20:27;color%20#fc8d6d;select%20:28;color%20#fb7858;select%20:29;color%20#fa6547;select%20:30;color%20#f44d38;select%20:31;color%20#e32f27;select%20:32;color%20#c9181d;select%20:33;color%20#ac1117;select%20:34;color%20#840711;select%20:35;color%20#75030f;select%20:36;color%20#75030f;select%20:37;color%20#75030f;select%20:38;

In [102]:
cust_mod = CustomRes("2crk","covalent_labels/ml.txt")
cust_mod.open_url(print_out=True)

{('156', 'A'): 30.58, ('158', 'A'): 32.79, ('304', 'A'): 34.39, ('138', 'A'): 39.46, ('366', 'A'): 30.31, ('372', 'A'): 32.4, ('107', 'A'): 34.04, ('307', 'A'): 37.93, ('141', 'A'): 37.81, ('14', 'A'): 38.05, ('32', 'A'): 34.72, ('128', 'A'): 30.21, ('224', 'A'): 32.15, ('24', 'A'): 31.02, ('105', 'A'): 33.61, ('108', 'A'): 34.05, ('103', 'A'): 32.65, ('15', 'A'): 38.04, ('25', 'A'): 35.5, ('298', 'A'): 37.17, ('16', 'A'): 34.94, ('116', 'A'): 36.39, ('369', 'A'): 35.11, ('381', 'A'): 30.87, ('129', 'A'): 31.86, ('223', 'A'): 33.48, ('140', 'A'): 34.26, ('242', 'A'): 34.49, ('303', 'A'): 34.23, ('101', 'A'): 39.78, ('147', 'A'): 30.75, ('170', 'A'): 35.15, ('11', 'A'): 37.19, ('172', 'A'): 34.29, ('125', 'A'): 30.57, ('40', 'A'): 34.35}
{('156', 'A'): '#79040f', ('158', 'A'): '#d11e1f', ('304', 'A'): '#f5523a', ('138', 'A'): '#fff0e8', ('366', 'A'): '#6b010e', ('372', 'A'): '#c4161c', ('107', 'A'): '#f14432', ('307', 'A'): '#fdccb8', ('141', 'A'): '#fdc9b3', ('14', 'A'): '#fdd0bc', ('3